In [44]:
import pandas as pd
import numpy as np
import pandas as pd
import re
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
import nltk
from nltk.corpus import stopwords
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim
import warnings
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import numpy as np 
import pandas as pd
import string
import re
from tqdm._tqdm_notebook import tqdm_notebook,tnrange,tqdm
from collections import Counter,OrderedDict
from gensim import models,corpora
import warnings
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.gensim
import gensim.models.phrases as gen
from gensim.models.coherencemodel import CoherenceModel
import re
import string
import numpy as np
import spacy
import random
import pandas as pd
from tqdm import tqdm
from collections import Counter
import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
review_data= pd.read_csv("scrape.csv")
print(review_data.head(2))
print(len(review_data))
print('Question_Text')
print(len(review_data.groupby('Question_Text')))
print('Token_Text')
print(len(review_data.groupby('Token_Text')))


   Unnamed: 0            Paper_ID Question_ID Question_Type  \
0           0  0460_m17_qp_22.pdf           1          MAIN   
1           1  0460_m17_qp_22.pdf         (a)           SUB   

                                       Question_Text  \
0  Study the map extract for Ballyvaghan, Ireland...   
1  The map has blue grid lines which make squares...   

                                          Token_Text  Word_Count  
0                                  map extract scale          71  
1  map lines squares area land grid square show a...         126  
2655
Question_Text
2476
Token_Text
2020


In [6]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [7]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to /Users/chae/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# To find text length boundaries
review_data.dropna(axis=0, how='any', inplace=True)

review_data['Question_Text'] = review_data['Question_Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Question_Text'].apply(lambda x: len(str(x).split()))

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >= 20)
df_within = review_data[mask]

df_sampled = df_within.groupby('Paper_ID').apply(lambda x: x.sample(n=min(20000, len(x)))).reset_index(drop=True)

print('Number of questions with words within given range:')
print(len(df_within))


Number of questions with words within given range:
301


In [64]:
# Drop rows with missing values
review_data.dropna(axis=0, how='any', inplace=True)
pattern = r'km\d*'
# Clean text and calculate the number of words in each question
review_data['Question_Text'] = review_data['Question_Text'].apply(clean_text)
store = review_data['Question_Text']
# review_data['Contains_km'] = review_data['Question_Text'].apply(lambda x: 'km' in str(x).lower())
review_data['Contains_km'] = review_data['Question_Text'].apply(lambda x: bool(re.search(pattern, str(x).lower())))

# Filter questions containing 'km'
df_containing_km = review_data[review_data['Contains_km']]

print('Number of questions containing "km":')
print(len(df_containing_km))


Number of questions containing "km":
6


In [42]:
test_str = 'hi i am chae'

letter = "am"

res = len([ele for ele in test_str.split() if letter in ele])

print("Words count : " + str(res))

Words count : 1


In [47]:
ALPHA = 0.1
BETA = 0.1
NUM_TOPICS = 20
sp = spacy.load("en_core_web_sm")

np.random.seed(42)
random.seed(42)


In [68]:
def generate_frequencies(review_data, max_docs=10000):
    freqs = Counter()
    all_stopwords = sp.Defaults.stop_words
    all_stopwords.add('enron')
    nr_tokens = 0

    for doc in review_data[:max_docs]:
        tokens = sp.tokenizer(doc)
        for token in tokens:
            token_text = token.text.lower()
            if token_text not in all_stopwords and token.is_alpha:
                nr_token += 1
                freqs[token_text] += 1
    return

def get_vocab(freqs, freq_threshold=3):
    vocab = {}
    vocab_idx_str = {}
    vocab_idx = 0

    for word in freqs:
        if freqs[words] >= freq_threshold:
            vocab[word] = vocab_idc
            vocab_idx_str[vocab_idx] = word

    return vocab, vocab_idx_str

def tokenize_dataset(review_data, vocab, max_docs=10000):
    nr_tokens = 0
    nr_docs = 0
    docs = []

    for doc in review_data[:max_docs]:
        tokens = sp.tokenizer(doc)

        if len(tokens) > 1:
            doc = []
            for token in tokens:
                token_text = token.text.lower()
                if token_text in vocab:
                    doc.append(token_text)
                    nr_tokens += 1
            nr_docs += 1
            docs.append(doc)

    print(f'Number of questions: {nr_docs}')
    print(f'Number of words: {nr_tokens}')

    corpus = []
    for doc in docs:
        corpus_d = []

        for token in doc:
            corpus_d.append(vocab[token])

        corpus.append(np.asarray(corpus_d))

    return docs, corpus

In [71]:
data = review_data["Question_Text"].sample(frac=0.5, random_state=42).values
freqs = generate_frequencies(data, max_docs=10000)
vocab, vocab_idx_str = get_vocab(freqs)
docs, corpus = tokenize_dataset(data, vocab)
vocab_size = len(vocab)

UnboundLocalError: cannot access local variable 'nr_token' where it is not associated with a value

In [62]:
import re

# Define a regex pattern to match variations of "km"
pattern = r'km\d*'  # This matches "km" followed by zero or more digits (\d*)

# Apply the regex pattern to each question text
review_data['Contains_km'] = review_data['Question_Text'].apply(lambda x: bool(re.search(pattern, str(x).lower())))

# Filter questions containing the pattern
df_containing_km = review_data[review_data['Contains_km']]

print('Number of questions containing "km" or "km2", etc.:')
print(len(df_containing_km))


Number of questions containing "km" or "km2", etc.:
6


In [70]:
print(sp.Defaults.stop_words)

{'toward', 'various', 'her', 'i', 'towards', "n't", 'such', 'so', 'whereas', 'no', 'across', 'the', 'thence', 'has', 'call', 'unless', 'how', 'in', 'via', "'ll", 'eleven', 'two', 'more', 'keep', 'get', "'s", 'if', 'yourselves', 'n‘t', 'other', 'whether', "'m", "'re", 'been', 'anywhere', 'those', 'onto', 'please', 'all', 'our', 'some', 'well', 'five', 'us', 'therein', 'at', 'nevertheless', '‘re', 'under', 'besides', 'many', 'almost', 'over', 'during', 'fifty', 'their', 'neither', 'thereupon', 'by', 'above', 'three', 'could', 'thereafter', 'since', '’ll', 'never', 'ourselves', 'either', 'an', 'was', 'too', 'something', 'put', 'are', 'for', 'therefore', 'also', 'however', 'both', 'perhaps', 'wherever', 'it', 'themselves', 'himself', 'its', 'go', 'up', 'even', 'my', 'sometimes', 'we', 're', 'than', 'you', '’ve', 'must', 'rather', 'top', 'most', 'few', 'along', 'show', 'wherein', '‘m', 'though', "'ve", 'off', 'fifteen', 'yours', 'now', 'much', 'hers', 'enough', 'alone', 'otherwise', 'somewh